#<center> Fake news Detection </center>


In [0]:
#libraries
import numpy as np
import pandas as pd
from numpy import genfromtxt
from google.colab import files

#text data config : nltk library
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools

#to_plot_graph 
import matplotlib.pyplot as plt



In [3]:
#Dataset
uploaded = files.upload()
data = pd.read_csv("fakenews.csv")
#print(data)
text = data.text
label = data.label
x_train, x_test, y_train, y_test = train_test_split(text,label,test_size=0.30)

#feature extraction

countvectorizer = CountVectorizer()
count_train = countvectorizer.fit_transform(x_train)
count_test = countvectorizer.transform(x_test)

tfidf_vector = TfidfVectorizer(max_df=0.7)
tfidf_train = tfidf_vector.fit_transform(x_train)
tfidf_test = tfidf_vector.transform(x_test)

feature_arr = tfidf_vector.get_feature_names()

tfidf_df = pd.DataFrame(tfidf_train.A, columns  = tfidf_vector.get_feature_names())

#print(tfidf_df)



Saving fakenews.csv to fakenews.csv


In [0]:
#Plotting graph
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
   
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('')
#         print('Confusion matrix, with little normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.rcParams['figure.figsize'] = [10, 10]
    plt.show()


In [7]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /content/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
#stemmer
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)
def stem_func(text):
    words = word_tokenize(text)
    str1 = "";
    for w in words:
        str1 = str1 + " " + stemmer.stem(w)
    return str1
  
news = data['text']
y = data.label 
index_array = y.index.values
verdict = []
stemmed_news = []
for index in index_array:
    val = y.at[index]
    verdict.append(val)
    stemmed_n = stem_func(news.at[index])
    stemmed_news.append(stemmed_n)
   

In [0]:
X_train, X_test, y_train, y_test = train_test_split(stemmed_news, verdict, test_size=0.4, random_state=53)


tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7) 
tfidf_train = tfidf_vectorizer.fit_transform(X_train) 
tfidf_test = tfidf_vectorizer.transform(X_test)


In [14]:
#classifier
#naive bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
clf_nav = MultinomialNB()
clf_nav = GaussianNB()
clf_nav.fit(tfidf_train.todense(),y_train)
# nav_predict = clf_nav.predict(tfidf_test.todense())
#print(clf_nav.predict(tfidf_test[1].todense()))
#print(type(y_test))
test_score = accuracy_score(nav_predict,y_test)
print(test_score*100)
#print(confusion_mat)
nav_predict = clf_nav.predict(tfidf_test.todense())
confusion_mat = confusion_matrix(y_test, nav_predict, labels=['FAKE', 'REAL'])
plot_confusion_matrix(confusion_mat, 
                      classes=['FAKE', 'REAL'], 
                      title="Confusion Matrix for \n Naive Bayes Classifier",
                      cmap = plt.confusion_mat.Greys
                     )



78.88713496448302
[[ 956  280]
 [ 255 1043]]


In [0]:
#classifier
#svm
from sklearn.svm import SVC
clf_svm = SVC()
clf_svm.fit(tfidf_train.todense(),y_train)
svm_predict = clf_svm.predict(tfidf_test.todense())
# print(clf_svm.predict(tfidf_test[1].todense()))
# print(list(y_test)[1])
test_score = accuracy_score(svm_predict,y_test)
print(test_score*100)

svm_predict = clf_svm.predict(tfidf_test.todense())
confusion_mat = confusion_matrix(y_test, svm_predict, labels=['FAKE', 'REAL'])
plot_confusion_matrix(confusion_mat, 
                      classes=['FAKE', 'REAL'], 
                      title="Confusion Matrix for \n Support Vector Machine Classifier ",
                      cmap = plt.confusion_mat.Greens
                     )

In [0]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
clf_tree= DecisionTreeClassifier()
clf_tree.fit(tfidf_train.todense(),y_train)
print(clf_tree.predict(tfidf_test[1].todense()))
print(list(y_test)[1])
test_score = accuracy_score(clf_tree.predict(tfidf_test.todense()),y_test)
print(test_score1*100)

Dt_predict = clf_tree.predict(tfidf_test.todense())
confusion_mat = confusion_matrix(y_test, Dt_predict, labels=['FAKE', 'REAL'])
plot_confusion_matrix(confusion_mat, 
                      classes=['FAKE', 'REAL'], 
                      title="Confusion Matrix for \n Decision Tree Classifier ",
                      cmap = plt.confusion_mat.Blues
                     )

In [0]:
#Neural Network
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(tfidf_train.todense(),y_train)
print(clf.predict(tfidf_test[1].todense()))
print(list(y_test)[1])
test_score1 = accuracy_score(clf.predict(tfidf_test.todense()),y_test)
print(test_score1*100)

nn_predict = clf.predict(tfidf_test.todense())
confusion_mat = confusion_matrix(y_test, nn_predict, labels=['FAKE', 'REAL'])
plot_confusion_matrix(confusion_mat, 
                      classes=['FAKE', 'REAL'], 
                      title="Confusion Matrix for \n Multi Layer Perceptron ",
                      cmap = plt.confusion_mat.Reds
                     )

#<center> Documentation </center>
Graph Doumentation See full source and example: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html